<a href="https://colab.research.google.com/github/ayobamiakomolafe/Zindi-projects/blob/master/MachineHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**An algorithm to predict the price of retail items belonging to different categories**

In [ ]:
Test='/content/drive/My Drive/Test.csv'
Train='/content/drive/My Drive/Train.csv'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train=pd.read_csv(Train)
df_test=pd.read_csv(Test)

In [ ]:
df_train.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


 Date-Time, Feature Engineering

In [ ]:
df_train['InvoiceDate']=df_train['InvoiceDate'].astype('datetime64')
df_train['dayofweek']=df_train['InvoiceDate'].dt.dayofweek
df_train['month']=df_train['InvoiceDate'].dt.month
df_train['hour']=df_train['InvoiceDate'].dt.hour
df_train['weekend']=df_train['dayofweek'] > 4
df_train['weekend'].replace(False, 0,inplace=True)
df_train['weekend'].replace(True, 1,inplace=True)

In [ ]:
X=df_train.drop(['UnitPrice','InvoiceDate'], axis=1)

In [ ]:
y=df_train['UnitPrice']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


from sklearn.preprocessing import StandardScaler
x_train,x_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=0)
sc= StandardScaler()


gb=GradientBoostingRegressor()
lr=LinearRegression()
rf=RandomForestRegressor(n_estimators=400)
poly=PolynomialFeatures(degree=3) # best degree is 3 with lr, doesnt work with others


Test file

In [ ]:
df_test=pd.read_csv(Test)

In [ ]:
df_test.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country
0,3139,1709,1079,3,2011-02-22 15:22:00,16161.0,35
1,617,510,3457,1,2010-12-08 12:46:00,17341.0,35
2,14653,604,694,36,2011-10-25 13:53:00,15158.0,35
3,8634,1478,3473,2,2011-06-27 12:38:00,16033.0,35
4,15546,3216,871,1,2011-11-06 16:14:00,15351.0,35


In [ ]:
df_test['InvoiceDate']=df_test['InvoiceDate'].astype('datetime64')

In [ ]:
df_test['dayofweek']=df_test['InvoiceDate'].dt.dayofweek
df_test['month']=df_test['InvoiceDate'].dt.month
df_test['hour']=df_test['InvoiceDate'].dt.hour
df_test['weekend']=df_test['dayofweek'] > 4
df_test['weekend'].replace(False, 0,inplace=True)
df_test['weekend'].replace(True, 1,inplace=True)


In [ ]:
df_test=df_test.drop('InvoiceDate', axis=1)

Model train

In [ ]:
!pip install lightgbm

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lg=LGBMRegressor()
gb=GradientBoostingRegressor()
rf=RandomForestRegressor(n_estimators=400)

In [ ]:
X_sc=sc.fit_transform(X)
df_test_sc=sc.transform(df_test)

In [ ]:
from sklearn.ensemble import VotingRegressor
vf=VotingRegressor([('rf', rf), ('lg', lg), ('gb', gb)], weights=[0.50,0.20,0.27])
vf.fit(X_sc, y)

In [ ]:
price=vf.predict(df_test_sc)

In [ ]:
df_price=pd.DataFrame(price)

In [ ]:
df_price=df_price.rename(columns={0: "UnitPrice"})

In [ ]:
df_price.shape

(122049, 1)

In [ ]:
from google.colab import files
df_price.to_csv('./vfh.csv', sep = ',', index = False)

In [ ]:
files.download('vfh.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>